# Training two regression models to explore the performance.

This models are latter relabeled into two values:
> 1 = 'Sad'
>
> 0 = 'Happy'

The perfromance of the re-labeled outputs is reported.

In [0]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot  as plt
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
!ls gdrive/My\ Drive/Colab\ Notebooks

 DF_complete_nona.pickle		     X_train_scaled.pickle
 DF_R00_DND_WITHOUT_HR_STEPS.pickle	     X_unknown_test_scaled.pickle
 DF_Sicong.csv				     y_median.pickle
 HR_R00_DND.pickle			     y.pickle
 subjects.pickle			     y_test_know.pickle
'Using_JIT[Just in Time]_with_Numba.ipynb'   y_test_unknow.pickle
 X_known_test_scaled.pickle		     y_train.pickle
 X.pickle


## Ingest data

In [0]:
#Load my DF
obj = ['DF_complete_nona', 'X', 'y', 'y_median', 'subjects']

for i in obj:
    with open(f'./gdrive/My Drive/Colab Notebooks/{i}.pickle','rb') as file:
        exec(i + " = pickle.load(file)")  # Trucazo.  I am creating variables names using my list of strings

In [20]:
X.head()

,HAN_actual,HAN_ideal,HAP_actual,HAP_ideal,HA_actual,HA_ideal,LAN_actual,LAN_ideal,LAP_actual,LAP_ideal,LA_actual,LA_ideal,N_actual,N_ideal,P_actual,P_ideal,Age,Children,Household_income,BMI,BAS_D,BAS_FS,BAS_RR,BIS.5,BIS_total,NS_total,Conscientiousness_scaled,Extraversion_scaled,Neuroticism_scaled,SWLS,FTP,SBQ,Minutes Asleep,Minutes Awake,Number of Awakenings,Time in Bed,Sex_Female,Sex_Male,Ethnicity_Asian,Ethnicity_Biracial,...,hr_mean_30m,hr_var_30m,hr_std_30m,hr_median_30m,hr_rest_rate_30m,hr_moderate_rate_30m,hr_very_active_rate_30m,SDNN_10m,pHR2_10m,rMSSD_10m,low_hr_10m,high_hr_10m,l_h_10m,CR_10m,hr_mean_10m,hr_var_10m,hr_std_10m,hr_median_10m,hr_rest_rate_10m,hr_moderate_rate_10m,hr_very_active_rate_10m,SDNN_5m,pHR2_5m,rMSSD_5m,low_hr_5m,high_hr_5m,l_h_5m,CR_5m,hr_mean_5m,hr_var_5m,hr_std_5m,hr_median_5m,hr_rest_rate_5m,hr_moderate_rate_5m,hr_very_active_rate_5m,hr_0,hr_0.3,hr_0.5,hr_0.8,hr_1
0,1.333333,1.0,2.6,2.8,2.0,2.333333,1.666667,1.0,3.6,4.2,2.0,1.8,1.0,1.0,4.333333,4.333333,43.0,3.0,9.0,27.328927,14.0,16.0,23.0,24.0,51.0,11.0,0.875,0.583333,0.208333,25.0,5.5,60.0,147.0,10.0,6.0,157.0,1,0,0,0,...,78.654545,37.800665,6.148225,77.0,0.539394,0.121212,0.000000,0.001125,0.186441,1.978700,71.0,99.0,0.717172,0.779528,81.916667,61.806497,7.861711,80.0,0.316667,0.233333,0.000000,0.001094,0.125000,1.968502,76.0,99.0,0.767677,0.779528,85.757576,68.189394,8.257687,82.0,0.181818,0.424242,0.000000,63.0,77.0,85.0,99.0,127.0
1,1.333333,1.0,2.6,2.8,2.0,2.333333,1.666667,1.0,3.6,4.2,2.0,1.8,1.0,1.0,4.333333,4.333333,43.0,3.0,9.0,27.328927,14.0,16.0,23.0,24.0,51.0,11.0,0.875,0.583333,0.208333,25.0,5.5,60.0,317.0,40.0,17.0,357.0,1,0,0,0,...,60.913514,150.818566,12.280821,57.0,0.675676,0.151351,0.075676,0.003763,0.188406,3.180557,50.0,103.0,0.485437,0.811024,66.628571,325.019462,18.028296,56.0,0.557143,0.342857,0.200000,0.000643,0.035714,1.224745,50.0,58.0,0.862069,0.456693,53.034483,3.463054,1.860928,53.0,1.000000,0.000000,0.000000,42.0,58.0,65.0,88.0,127.0
2,1.333333,1.0,2.6,2.8,2.0,2.333333,1.666667,1.0,3.6,4.2,2.0,1.8,1.0,1.0,4.333333,4.333333,43.0,3.0,9.0,27.328927,14.0,16.0,23.0,24.0,51.0,11.0,0.875,0.583333,0.208333,25.0,5.5,60.0,317.0,40.0,17.0,357.0,1,0,0,0,...,88.798283,154.722066,12.438732,86.0,0.000000,0.952790,0.446352,0.001028,0.166667,2.455153,66.0,91.0,0.725275,0.716535,77.589041,38.745434,6.224583,77.0,0.000000,0.849315,0.054795,0.001182,0.181818,2.037527,66.0,91.0,0.725275,0.716535,76.852941,51.947415,7.207456,75.0,0.000000,0.764706,0.117647,42.0,61.0,70.0,88.0,127.0
3,1.333333,1.0,2.6,2.8,2.0,2.333333,1.666667,1.0,3.6,4.2,2.0,1.8,1.0,1.0,4.333333,4.333333,43.0,3.0,9.0,27.328927,14.0,16.0,23.0,24.0,51.0,11.0,0.875,0.583333,0.208333,25.0,5.5,60.0,419.0,34.0,17.0,453.0,1,0,0,0,...,71.228856,10.887363,3.299600,71.0,0.000000,0.885572,0.000000,0.000624,0.027778,1.201850,65.0,82.0,0.792683,0.625954,72.178082,10.870624,3.297063,72.0,0.000000,0.931507,0.000000,0.000522,0.054054,1.414214,69.0,82.0,0.841463,0.625954,74.000000,8.648649,2.940858,73.5,0.000000,1.000000,0.000000,42.0,58.0,67.0,87.0,131.0
4,1.333333,1.0,2.6,2.8,2.0,2.333333,1.666667,1.0,3.6,4.2,2.0,1.8,1.0,1.0,4.333333,4.333333,43.0,3.0,9.0,27.328927,14.0,16.0,23.0,24.0,51.0,11.0,0.875,0.583333,0.208333,25.0,5.5,60.0,419.0,34.0,17.0,453.0,1,0,0,0,...,85.583784,77.168214,8.784544,82.0,0.000000,1.000000,0.194595,0.000255,0.000000,0.820413,79.0,85.0,0.929412,0.648855,81.320755,2.914369,1.707152,81.0,0.000000,1.000000,0.000000,0.000252,0.000000,0.721110,79.0,84.0,0.940476,0.641221,81.846154,2.935385,1.713296,82.5,0.000000,1.000000,0.000000,42.0,60.0,71.0,94.0,131.0


## Functions


In [0]:
# Importing useful things
from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import GroupShuffleSplit
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, mean_absolute_error,confusion_matrix,\
                            roc_auc_score, roc_curve,precision_recall_curve,precision_score,\
                            recall_score, accuracy_score, f1_score

#I define gss
gss = GroupShuffleSplit(n_splits=10, train_size=.7, random_state=10)

In [0]:
#define useful functions
def evaluate(model,
             test_set_name = [],
             x_test = [],
             y_test = []):
    '''
    Evaluate MAE, MSE,  RMSE of a model for all the test sets
    '''
    #initialize columns
    df = pd.DataFrame(columns = ['Test set', 'MAE', 'MSE', 'RMSE'])
    for ts , x , y in zip(test_set_name, x_test, y_test):
        predicted = model.predict(x)
        rmse  = np.sqrt(mean_squared_error(y, predicted))
        mae =  mean_absolute_error(y,predicted)
        mse = mean_squared_error(y, predicted)
        df = df.append({'Test set' : ts,
                        'MAE' : mae,
                        'MSE' : mse,
                        'RMSE' : rmse}, ignore_index= True )
    return df

def to_category (series_original):
    '''
    Transform the predicted value into a binary class
    1 = sad
    0 = happy
    '''
    series = series_original.copy()
    series[series_original >= 0] = 0 #happy is  0
    series[series_original < 0] = 1  #sad is 1
    return series

def conf_mat_rate(predict, real):
    '''
    Nice conf matrix
    '''
    tn, fp, fn, tp = confusion_matrix(predict, real).ravel()
    conf = pd.DataFrame(index= ['+','-', 'Total real'], columns= ['POS' , 'NEG'] , data = np.array([[tp,fn],[fp,tn],[1,1]]))
    conf.insert(2, 'Total predicted', np.array([tp+fn,fp+tn,0]))
    conf.iloc[0,0:2] = conf.iloc[0,0:2]/conf.iloc[0,2] # I calculate the rates
    conf.iloc[1,0:2] = conf.iloc[1,0:2]/conf.iloc[1,2]
    conf.iloc[2,:] = np.array([tp+fp,fn+tn,tp+fn+fp+tn]).astype(str) # I calculate the total real observations
    conf.iloc[2,:] = conf.iloc[2,:].astype(int) #formatting to integer
    return conf

def binary_class(model, dfs = None):
    '''
    Generate several metrics
    dfs : list of tuples. (X,y,name of DF)
          X = test X
          y = real labels
    '''
    #in this case I am testing with several DF only to check. In the final stage I would like to do a NestedCV
    if dfs is None:
        dfs = []
  
    all_values = {}
    for df in dfs:
        #df[0] -> X
        #df[1] -> y
        pred = to_category(model.predict(df[0]))
        real = to_category(df[1])
        cf = confusion_matrix(pred,real)
        cf_rate = conf_mat_rate(pred,real)
        
        metrics = {} #metric dictionary
        metrics_names = ['recall','precision','accuracy','f1']
        recall = recall_score(pred,real) #sensitivity
        precision = precision_score(pred,real)
        accuracy = accuracy_score(pred,real)
        f1 = f1_score(pred,real)
        for m in metrics_names:
            metrics[m] = eval(m)
        #append all
        all_values[df[2]] = [cf , cf_rate, metrics]
    return all_values

def metric_table (all_values, name_df, model_name):
    metric_df = pd.DataFrame(all_values[name_df][2], index=[model_name])
    return metric_df

def all_models_scores (models , models_names, name_df):
    '''
    one table per test df
    models : list
    models_name : list
    name_df : string
    '''
    metrics_names = ['recall','precision','accuracy','f1']
    df = pd.DataFrame(index = models_names, columns= metrics_names)
    
    for mod,name in zip(models, models_names):
        all_vals = binary_class(mod)
        df.loc[name,:] = metric_table(all_vals, name_df, model_name=name).values
    return df


def group_shuffle(model,X,y, gss):
    '''
    Function performs a group shuffle CV
    '''
    n = 0
    df = pd.DataFrame(columns = ['Test set',	'MAE',	'MSE',	'RMSE'])
    df2 = pd.DataFrame(columns=  ['recall',	'precision',	'accuracy',	'f1'])
    confusions = {}
    #Loop to train all the different combinations
    for train_idx, test_idx in gss.split(X, y, subjects):
        n += 1
        SCALER = preprocessing.StandardScaler().fit(X.iloc[train_idx,:]) # I would SCALE the data, but to dont contaminate the Fold I should scale using the training for every fold
        x_train = pd.DataFrame(data = SCALER.transform(X.iloc[train_idx,:]), columns = X.columns) # Apply scaler to train data
        x_test = pd.DataFrame(data = SCALER.transform(X.iloc[test_idx,:]), columns = X.columns) # Apply scaler to test data
        model = model.fit(x_train, y.iloc[train_idx])
        df = df.append(evaluate(model,
                                 test_set_name=[f'Set {n}'],
                                 x_test = [x_test],
                                 y_test= [y.iloc[test_idx]]
                               )
                      )
        all_values = binary_class(model, dfs = [(x_test, y.iloc[test_idx], f'Set {n}') ]) 
        confusions[f'Set {n}'] = all_values[f'Set {n}'][1]
        df2 = df2.append(pd.DataFrame(all_values[f'Set {n}'][-1], index =[n]))
    return df,df2, confusions

In [0]:
#Function to evaluate different metrics for the model MERF
def evaluate_MERF(model,
             test_set_name = [],
             x_test = [],
             y_test = [],
             Z = [],
             clusters = []):
    '''
    Evaluate MAE, MSE,  RMSE of a model for all the test sets
    '''
    #initialize columns
    df = pd.DataFrame(columns = ['Test set', 'MAE', 'MSE', 'RMSE'])
    
    for ts , x , y, z, cluster in zip(test_set_name, x_test, y_test, Z, clusters):
        predicted = model.predict(x, z, cluster)
        rmse  = np.sqrt(mean_squared_error(y, predicted))
        mae =  mean_absolute_error(y,predicted)
        mse = mean_squared_error(y, predicted)
        df = df.append({'Test set' : ts,
                        'MAE' : mae,
                        'MSE' : mse,
                        'RMSE' : rmse}, ignore_index= True )
    return df


In [0]:

#need to work on this one
def group_shuffle_MERF(model,X,y,Z,cluster,gss):
    '''
    Function performs a group shuffle CV
    '''
    n = 0
    df = pd.DataFrame(columns = ['Test set',	'MAE',	'MSE',	'RMSE'])
    df2 = pd.DataFrame(columns=  ['recall',	'precision',	'accuracy',	'f1'])
    confusions = {}
    #Loop to train all the different combinations
    for train_idx, test_idx in gss.split(X, y, subjects):
        n += 1
        SCALER = preprocessing.StandardScaler().fit(X.iloc[train_idx,:]) # I would SCALE the data, but to dont contaminate the Fold I should scale using the training for every fold
        x_train = pd.DataFrame(data = SCALER.transform(X.iloc[train_idx,:]), columns = X.columns) # Apply scaler to train data
        x_test = pd.DataFrame(data = SCALER.transform(X.iloc[test_idx,:]), columns = X.columns) # Apply scaler to test data
        model = model.fit(x_train, y.iloc[train_idx])
        df = df.append(evaluate(model,
                                 test_set_name=[f'Set {n}'],
                                 x_test = [x_test],
                                 y_test= [y.iloc[test_idx]]
                               )
                      )
        all_values = binary_class(model, dfs = [(x_test, y.iloc[test_idx], f'Set {n}') ]) 
        confusions[f'Set {n}'] = all_values[f'Set {n}'][1]
        df2 = df2.append(pd.DataFrame(all_values[f'Set {n}'][-1], index =[n]))
    return df,df2, confusions

## Model Parameters

In [0]:
model_rf = RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=140, max_features='sqrt', max_leaf_nodes=None,
                                 min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=4, min_samples_split=4,
                                 min_weight_fraction_leaf=0.0, n_estimators=1500, n_jobs=None, oob_score=False, random_state=None,
                                 verbose=0, warm_start=False)

model_bayesian_ridge = BayesianRidge(alpha_1=1e-05, alpha_2=1e-09, compute_score=False, copy_X=True, fit_intercept=True,
                                     lambda_1=1e-10, lambda_2=0.0001, n_iter=300, normalize=False, tol=0.001, verbose=False)

model_SVR = svm.SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.001, kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

Fit a MERF model for Known subjects
.

.

.

.



## Fitting models for Unknown (new) Subjects

### Y

#### Bayesian Ridge

In [0]:
# I am going to GroupShuffle CV
# For every run I will get predictions
# I will binarize this predictions and I will get some accuracy metrics

### Bayesian Ridge
reg_metrics_bayes, predict_metric_bayes, all_values_bayes = group_shuffle(model_bayesian_ridge, X, y, gss)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
reg_metrics_bayes.mean()

MAE     0.969468
MSE     1.579867
RMSE    1.253791
dtype: float64

In [0]:
#Proportion of 'Sad cases' on 1
to_category(y).sum()/y.shape[0]

0.10857709469509172

In [0]:
predict_metric_bayes.mean()

recall       0.182947
precision    0.018863
accuracy     0.855844
f1           0.030501
dtype: float64

In [0]:
all_values_bayes

{'Set 1':                   POS       NEG Total predicted
 +            0.173913  0.826087              23
 -           0.0959119  0.904088             636
 Total real         65       594             659,
 'Set 10':                  POS       NEG Total predicted
 +                NaN       NaN               0
 -           0.122449  0.877551             637
 Total real        78       559             637,
 'Set 2':                  POS       NEG Total predicted
 +                  0         1               4
 -           0.153344  0.846656             613
 Total real        94       523             617,
 'Set 3':                  POS       NEG Total predicted
 +                0.1       0.9              10
 -           0.075643  0.924357             661
 Total real        51       620             671,
 'Set 4':                  POS       NEG Total predicted
 +           0.166667  0.833333              12
 -           0.148649  0.851351             666
 Total real       101       577   

#### Random Forest

In [0]:
### Random Forest
reg_metrics_rf, predict_metric_rf, all_values_rf = group_shuffle(model_rf, X, y, gss)
reg_metrics_rf.mean()

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


MAE     0.892272
MSE     1.285793
RMSE    1.133163
dtype: float64

In [0]:
predict_metric_rf.mean()

recall       0.066667
precision    0.003217
accuracy     0.888112
f1           0.005935
dtype: float64

In [0]:
predict_metric_rf

,recall,precision,accuracy,f1
1,0.000000,0.000000,0.860390,0.000000
2,0.000000,0.000000,0.867365,0.000000
3,0.000000,0.000000,0.904437,0.000000
4,0.166667,0.020408,0.913115,0.036364
5,0.000000,0.000000,0.918660,0.000000
6,0.500000,0.011765,0.871017,0.022989
7,0.000000,0.000000,0.872814,0.000000
8,0.000000,0.000000,0.892063,0.000000
9,0.000000,0.000000,0.893058,0.000000
10,0.000000,0.000000,0.888199,0.000000


In [0]:
all_values_rf

{'Set 1':                  POS       NEG Total predicted
 +                  0         1               1
 -           0.138211  0.861789             615
 Total real        85       531             616,
 'Set 10':                 POS      NEG Total predicted
 +                 0        1               1
 -           0.11042  0.88958             643
 Total real       71      573             644,
 'Set 2':                  POS       NEG Total predicted
 +                  0         1               2
 -           0.129597  0.870403             571
 Total real        74       499             573,
 'Set 3':                   POS       NEG Total predicted
 +                 NaN       NaN               0
 -           0.0955631  0.904437             586
 Total real         56       530             586,
 'Set 4':                   POS       NEG Total predicted
 +            0.166667  0.833333               6
 -           0.0794702   0.92053             604
 Total real         49       561       

#### SVM

In [0]:
### SVM Regression
reg_metrics_SVR, predict_metric_SVR, all_values_SVR = group_shuffle(model_SVR, X, y, gss)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
reg_metrics_SVR

,Test set,MAE,MSE,RMSE
0,Set 1,0.949686,1.510700,1.229105
0,Set 2,1.020616,1.773214,1.331621
0,Set 3,0.825036,1.188461,1.090166
0,Set 4,0.979680,1.647981,1.283737
0,Set 5,0.942568,1.469517,1.212236
0,Set 6,1.020234,1.766681,1.329166
0,Set 7,0.957827,1.554444,1.246773
0,Set 8,1.233239,2.506148,1.583082
0,Set 9,0.991759,1.546247,1.243482
0,Set 10,0.938787,1.544244,1.242676


In [0]:
reg_metrics_SVR.mean()

MAE     0.985943
MSE     1.650764
RMSE    1.279204
dtype: float64

In [0]:
predict_metric_SVR

,recall,precision,accuracy,f1
1,0.074074,0.030769,0.866464,0.043478
2,0.083333,0.010638,0.831442,0.018868
3,0.181818,0.039216,0.913562,0.064516
4,0.000000,0.000000,0.851032,0.000000
5,0.066667,0.013889,0.868217,0.022989
6,0.400000,0.019802,0.834684,0.037736
7,0.250000,0.100000,0.866419,0.142857
8,0.000000,0.000000,0.778125,0.000000
9,0.111111,0.010753,0.833611,0.019608
10,0.500000,0.012821,0.877551,0.025000


In [0]:
predict_metric_SVR.mean()

recall       0.166700
precision    0.023789
accuracy     0.852111
f1           0.037505
dtype: float64

### Y median

#### Bayesian Ridge

In [0]:
#Bayes Ridge
reg_metrics_bayes, predict_metric_bayes, all_values_bayes = group_shuffle(model_bayesian_ridge, X, y_median, gss)
reg_metrics_bayes.mean()

MAE     0.701709
MSE     0.864653
RMSE    0.929102
dtype: float64

In [0]:
#Proportion of 'Sad cases' on 1
to_category(y_median).sum()/y_median.shape[0]

0.4134853743182945

In [0]:
predict_metric_bayes.mean()

recall       0.411705
precision    0.974643
accuracy     0.415662
f1           0.578643
dtype: float64

In [0]:
all_values_bayes

{'Set 1':                  POS       NEG Total predicted
 +           0.415825  0.584175             594
 -                0.5       0.5              22
 Total real       258       358             616,
 'Set 10':                  POS       NEG Total predicted
 +                0.4       0.6             635
 -           0.555556  0.444444               9
 Total real       259       385             644,
 'Set 2':                  POS       NEG Total predicted
 +           0.431858  0.568142             565
 -              0.375     0.625               8
 Total real       247       326             573,
 'Set 3':                  POS       NEG Total predicted
 +           0.389474  0.610526             570
 -             0.4375    0.5625              16
 Total real       229       357             586,
 'Set 4':                  POS       NEG Total predicted
 +           0.405405  0.594595             592
 -           0.333333  0.666667              18
 Total real       246       364       

#### Random Forest

In [0]:
### Random Forest
reg_metrics_rf, predict_metric_rf, all_values_rf = group_shuffle(model_rf, X, y_median, gss)
reg_metrics_rf.mean()

MAE     0.743906
MSE     0.972680
RMSE    0.983964
dtype: float64

In [0]:
predict_metric_rf.mean()

recall       0.409253
precision    0.731588
accuracy     0.453681
f1           0.523919
dtype: float64

In [0]:
all_values_rf

{'Set 1':                  POS       NEG Total predicted
 +           0.414141  0.585859             495
 -           0.438017  0.561983             121
 Total real       258       358             616,
 'Set 10':                  POS       NEG Total predicted
 +           0.379921  0.620079             508
 -           0.485294  0.514706             136
 Total real       259       385             644,
 'Set 2':                  POS       NEG Total predicted
 +           0.444695  0.555305             443
 -           0.384615  0.615385             130
 Total real       247       326             573,
 'Set 3':                  POS       NEG Total predicted
 +           0.389868  0.610132             454
 -           0.393939  0.606061             132
 Total real       229       357             586,
 'Set 4':                  POS       NEG Total predicted
 +           0.392683  0.607317             410
 -              0.425     0.575             200
 Total real       246       364       

#### SVM

In [0]:
### SVM Regression
reg_metrics_SVR, predict_metric_SVR, all_values_SVR = group_shuffle(model_SVR, X, y_median, gss)

In [0]:
#Regression
reg_metrics_SVR.mean()

MAE     0.772726
MSE     1.052125
RMSE    1.023597
dtype: float64

In [0]:
#Predition
predict_metric_SVR.mean()

recall       0.422644
precision    0.487439
accuracy     0.503142
f1           0.451606
dtype: float64

## Fitting models for Known (old) Subjects

### Y

### Y median

# Sicong Data

In [0]:
fitbit_features = ['subject', "step_max", "step_min", "step_median", "steps_max_3h",
    "steps_min_3h", "steps_mean_3h", "steps_var_3h", "steps_median_3h",
    "move_rate_3h", "active_rate_3h", "very_active_rate_3h",
    "running_rate_3h", "steps_max_1h", "steps_min_1h", "steps_mean_1h",
    "steps_var_1h", "steps_median_1h", "move_rate_1h",
    "active_rate_1h", "very_active_rate_1h", "running_rate_1h",
    "steps_max_30m", "steps_min_30m", "steps_mean_30m",
    "steps_var_30m", "steps_median_30m", "move_rate_30m",
    "active_rate_30m", "very_active_rate_30m", "running_rate_30m",
    "steps_max_10m", "steps_min_10m", "steps_mean_10m",
    "steps_var_10m", "steps_median_10m", "move_rate_10m",
    "active_rate_10m", "very_active_rate_10m", "running_rate_10m",
    "steps_max_5m", "steps_min_5m", "steps_mean_5m", "steps_var_5m",
    "steps_median_5m", "move_rate_5m", "active_rate_5m",
    "very_active_rate_5m", "running_rate_5m", "hr_max", "hr_min",
    "hr_med", "hr_0", "hr_0.3", "hr_0.5", "hr_0.8", "hr_1", "SDNN_3h",
    "pHR2_3h", "rMSSD_3h", "low_hr_3h", "high_hr_3h", "l_h_3h",
    "CR_3h", "hr_mean_3h", "hr_var_3h", "hr_std_3h", "hr_median_3h",
    "hr_rest_rate_3h", "hr_moderate_rate_3h", "hr_very_active_rate_3h",
    "SDNN_1h", "pHR2_1h", "rMSSD_1h", "low_hr_1h", "high_hr_1h",
    "l_h_1h", "CR_1h", "hr_mean_1h", "hr_var_1h", "hr_std_1h",
    "hr_median_1h", "hr_rest_rate_1h", "hr_moderate_rate_1h",
    "hr_very_active_rate_1h", "SDNN_30m", "pHR2_30m", "rMSSD_30m",
    "low_hr_30m", "high_hr_30m", "l_h_30m", "CR_30m", "hr_mean_30m",
    "hr_var_30m", "hr_std_30m", "hr_median_30m", "hr_rest_rate_30m",
    "hr_moderate_rate_30m", "hr_very_active_rate_30m", "SDNN_10m",
    "pHR2_10m", "rMSSD_10m", "low_hr_10m", "high_hr_10m", "l_h_10m",
    "CR_10m", "hr_mean_10m", "hr_var_10m", "hr_std_10m",
    "hr_median_10m", "hr_rest_rate_10m", "hr_moderate_rate_10m",
    "hr_very_active_rate_10m", "SDNN_5m", "pHR2_5m", "rMSSD_5m",
    "low_hr_5m", "high_hr_5m", "l_h_5m", "CR_5m", "hr_mean_5m",
    "hr_var_5m", "hr_std_5m", "hr_median_5m", "hr_rest_rate_5m",
    "hr_moderate_rate_5m", "hr_very_active_rate_5m"]

In [5]:
ALL_DATA = pd.read_csv('gdrive/My Drive/Colab Notebooks/DF_Sicong.csv')
ALL_DATA.head()

,Unnamed: 0,subject,la_p,ha_p,ha_n,la_n,la,p,n,ha,step_max,step_min,step_median,steps_max_3h,steps_min_3h,steps_mean_3h,steps_var_3h,steps_median_3h,move_rate_3h,active_rate_3h,very_active_rate_3h,running_rate_3h,steps_max_1h,steps_min_1h,steps_mean_1h,steps_var_1h,steps_median_1h,move_rate_1h,active_rate_1h,very_active_rate_1h,running_rate_1h,steps_max_30m,steps_min_30m,steps_mean_30m,steps_var_30m,steps_median_30m,move_rate_30m,active_rate_30m,very_active_rate_30m,running_rate_30m,...,BAS_FS,BAS_RR,HAP_actual,P_actual,LAP_actual,LA_actual,LAN_actual,N_actual,HAN_actual,HA_actual,HAP_ideal,P_ideal,LAP_ideal,LA_ideal,LAN_ideal,N_ideal,HAN_ideal,HA_ideal,Children,Age,BMI,survey_hour,r2r_corr,Corr_z,max_n,max_p,valence_p,valence_n,valence_reg,valence,edu_hs,edu_ba,edu_ma,edu_phd,is_married,is_divorced,is_single,is_widowed,is_with_partner,income
0,0,1047,4.0,3.0,1.0,1.0,2.0,4.0,1.0,1.0,110.0,0.0,0.0,110.0,0.0,4.311111,209.813284,0.0,0.155556,0.100000,0.066667,0.044444,76.0,0.0,2.216667,133.325141,0.0,0.050000,0.033333,0.033333,0.033333,76.0,0.0,4.133333,260.947126,0.0,0.066667,0.066667,0.066667,0.066667,...,16.0,23.0,2.6,4.333333,3.6,2.0,1.666667,1.0,1.333333,2.0,2.8,4.333333,4.2,1.8,1.0,1.0,1.0,2.333333,3.0,43.0,27.328927,21.0,0.356312,0.372656,1.0,4.0,3.666667,1.000000,2.666667,0,0,0,1,0,1,0,0,0,0,9
1,1,1047,4.0,1.0,1.0,2.0,4.0,2.0,1.0,1.0,111.0,0.0,0.0,70.0,0.0,1.205556,60.074829,0.0,0.044444,0.033333,0.016667,0.011111,21.0,0.0,1.100000,17.922034,0.0,0.066667,0.066667,0.016667,0.000000,21.0,0.0,1.800000,30.648276,0.0,0.100000,0.100000,0.033333,0.000000,...,16.0,23.0,2.6,4.333333,3.6,2.0,1.666667,1.0,1.333333,2.0,2.8,4.333333,4.2,1.8,1.0,1.0,1.0,2.333333,3.0,43.0,27.328927,7.0,0.356312,0.372656,2.0,4.0,2.333333,1.333333,1.000000,0,0,0,1,0,1,0,0,0,0,9
2,2,1047,2.0,2.0,1.0,2.0,1.0,5.0,1.0,1.0,111.0,0.0,0.0,102.0,0.0,18.950000,417.265642,14.5,0.672222,0.577778,0.377778,0.238889,86.0,0.0,19.566667,434.148023,14.5,0.683333,0.583333,0.400000,0.233333,74.0,0.0,24.466667,484.533333,23.0,0.733333,0.666667,0.533333,0.333333,...,16.0,23.0,2.6,4.333333,3.6,2.0,1.666667,1.0,1.333333,2.0,2.8,4.333333,4.2,1.8,1.0,1.0,1.0,2.333333,3.0,43.0,27.328927,16.0,0.356312,0.372656,2.0,5.0,3.000000,1.333333,1.666667,0,0,0,1,0,1,0,0,0,0,9
3,3,1047,3.0,1.0,1.0,2.0,2.0,3.0,1.0,1.0,111.0,0.0,0.0,72.0,0.0,3.750000,124.613128,0.0,0.166667,0.105556,0.061111,0.038889,72.0,0.0,3.250000,193.648305,0.0,0.066667,0.050000,0.050000,0.050000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,16.0,23.0,2.6,4.333333,3.6,2.0,1.666667,1.0,1.333333,2.0,2.8,4.333333,4.2,1.8,1.0,1.0,1.0,2.333333,3.0,43.0,27.328927,8.0,0.356312,0.372656,2.0,3.0,2.333333,1.333333,1.000000,0,0,0,1,0,1,0,0,0,0,9
4,4,1047,3.0,3.0,1.0,1.0,2.0,4.0,1.0,2.0,111.0,0.0,0.0,87.0,0.0,26.577778,496.803973,22.5,0.766667,0.727778,0.550000,0.383333,87.0,0.0,19.100000,535.345763,14.0,0.550000,0.516667,0.366667,0.266667,31.0,0.0,2.900000,61.610345,0.0,0.133333,0.133333,0.066667,0.033333,...,16.0,23.0,2.6,4.333333,3.6,2.0,1.666667,1.0,1.333333,2.0,2.8,4.333333,4.2,1.8,1.0,1.0,1.0,2.333333,3.0,43.0,27.328927,13.0,0.356312,0.372656,1.0,4.0,3.333333,1.000000,2.333333,0,0,0,1,0,1,0,0,0,0,9


In [6]:
Y_TARGET = ALL_DATA['valence_p'].copy() # I dont know what target variable is Sicong using. Need to ask.
subjects = ALL_DATA.subject # Unique subjects
X_ALL_DATA= ALL_DATA.copy().drop(columns = ['Unnamed: 0','subject','valence_p','valence_n', 'valence_reg' , 'valence'])
X_ALL_DATA.head()

,la_p,ha_p,ha_n,la_n,la,p,n,ha,step_max,step_min,step_median,steps_max_3h,steps_min_3h,steps_mean_3h,steps_var_3h,steps_median_3h,move_rate_3h,active_rate_3h,very_active_rate_3h,running_rate_3h,steps_max_1h,steps_min_1h,steps_mean_1h,steps_var_1h,steps_median_1h,move_rate_1h,active_rate_1h,very_active_rate_1h,running_rate_1h,steps_max_30m,steps_min_30m,steps_mean_30m,steps_var_30m,steps_median_30m,move_rate_30m,active_rate_30m,very_active_rate_30m,running_rate_30m,steps_max_10m,steps_min_10m,...,NS_total,BIS_total,BIS.5,BAS_D,BAS_FS,BAS_RR,HAP_actual,P_actual,LAP_actual,LA_actual,LAN_actual,N_actual,HAN_actual,HA_actual,HAP_ideal,P_ideal,LAP_ideal,LA_ideal,LAN_ideal,N_ideal,HAN_ideal,HA_ideal,Children,Age,BMI,survey_hour,r2r_corr,Corr_z,max_n,max_p,edu_hs,edu_ba,edu_ma,edu_phd,is_married,is_divorced,is_single,is_widowed,is_with_partner,income
0,4.0,3.0,1.0,1.0,2.0,4.0,1.0,1.0,110.0,0.0,0.0,110.0,0.0,4.311111,209.813284,0.0,0.155556,0.100000,0.066667,0.044444,76.0,0.0,2.216667,133.325141,0.0,0.050000,0.033333,0.033333,0.033333,76.0,0.0,4.133333,260.947126,0.0,0.066667,0.066667,0.066667,0.066667,76.0,0.0,...,11.0,51.0,24.0,14.0,16.0,23.0,2.6,4.333333,3.6,2.0,1.666667,1.0,1.333333,2.0,2.8,4.333333,4.2,1.8,1.0,1.0,1.0,2.333333,3.0,43.0,27.328927,21.0,0.356312,0.372656,1.0,4.0,0,0,1,0,1,0,0,0,0,9
1,4.0,1.0,1.0,2.0,4.0,2.0,1.0,1.0,111.0,0.0,0.0,70.0,0.0,1.205556,60.074829,0.0,0.044444,0.033333,0.016667,0.011111,21.0,0.0,1.100000,17.922034,0.0,0.066667,0.066667,0.016667,0.000000,21.0,0.0,1.800000,30.648276,0.0,0.100000,0.100000,0.033333,0.000000,21.0,0.0,...,11.0,51.0,24.0,14.0,16.0,23.0,2.6,4.333333,3.6,2.0,1.666667,1.0,1.333333,2.0,2.8,4.333333,4.2,1.8,1.0,1.0,1.0,2.333333,3.0,43.0,27.328927,7.0,0.356312,0.372656,2.0,4.0,0,0,1,0,1,0,0,0,0,9
2,2.0,2.0,1.0,2.0,1.0,5.0,1.0,1.0,111.0,0.0,0.0,102.0,0.0,18.950000,417.265642,14.5,0.672222,0.577778,0.377778,0.238889,86.0,0.0,19.566667,434.148023,14.5,0.683333,0.583333,0.400000,0.233333,74.0,0.0,24.466667,484.533333,23.0,0.733333,0.666667,0.533333,0.333333,74.0,0.0,...,11.0,51.0,24.0,14.0,16.0,23.0,2.6,4.333333,3.6,2.0,1.666667,1.0,1.333333,2.0,2.8,4.333333,4.2,1.8,1.0,1.0,1.0,2.333333,3.0,43.0,27.328927,16.0,0.356312,0.372656,2.0,5.0,0,0,1,0,1,0,0,0,0,9
3,3.0,1.0,1.0,2.0,2.0,3.0,1.0,1.0,111.0,0.0,0.0,72.0,0.0,3.750000,124.613128,0.0,0.166667,0.105556,0.061111,0.038889,72.0,0.0,3.250000,193.648305,0.0,0.066667,0.050000,0.050000,0.050000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,11.0,51.0,24.0,14.0,16.0,23.0,2.6,4.333333,3.6,2.0,1.666667,1.0,1.333333,2.0,2.8,4.333333,4.2,1.8,1.0,1.0,1.0,2.333333,3.0,43.0,27.328927,8.0,0.356312,0.372656,2.0,3.0,0,0,1,0,1,0,0,0,0,9
4,3.0,3.0,1.0,1.0,2.0,4.0,1.0,2.0,111.0,0.0,0.0,87.0,0.0,26.577778,496.803973,22.5,0.766667,0.727778,0.550000,0.383333,87.0,0.0,19.100000,535.345763,14.0,0.550000,0.516667,0.366667,0.266667,31.0,0.0,2.900000,61.610345,0.0,0.133333,0.133333,0.066667,0.033333,0.0,0.0,...,11.0,51.0,24.0,14.0,16.0,23.0,2.6,4.333333,3.6,2.0,1.666667,1.0,1.333333,2.0,2.8,4.333333,4.2,1.8,1.0,1.0,1.0,2.333333,3.0,43.0,27.328927,13.0,0.356312,0.372656,1.0,4.0,0,0,1,0,1,0,0,0,0,9


In [7]:
X_FITBIT_DATA = X_ALL_DATA.copy().filter(fitbit_features)
X_FITBIT_DATA.head()

,step_max,step_min,step_median,steps_max_3h,steps_min_3h,steps_mean_3h,steps_var_3h,steps_median_3h,move_rate_3h,active_rate_3h,very_active_rate_3h,running_rate_3h,steps_max_1h,steps_min_1h,steps_mean_1h,steps_var_1h,steps_median_1h,move_rate_1h,active_rate_1h,very_active_rate_1h,running_rate_1h,steps_max_30m,steps_min_30m,steps_mean_30m,steps_var_30m,steps_median_30m,move_rate_30m,active_rate_30m,very_active_rate_30m,running_rate_30m,steps_max_10m,steps_min_10m,steps_mean_10m,steps_var_10m,steps_median_10m,move_rate_10m,active_rate_10m,very_active_rate_10m,running_rate_10m,steps_max_5m,...,rMSSD_30m,low_hr_30m,high_hr_30m,l_h_30m,CR_30m,hr_mean_30m,hr_var_30m,hr_std_30m,hr_median_30m,hr_rest_rate_30m,hr_moderate_rate_30m,hr_very_active_rate_30m,SDNN_10m,pHR2_10m,rMSSD_10m,low_hr_10m,high_hr_10m,l_h_10m,CR_10m,hr_mean_10m,hr_var_10m,hr_std_10m,hr_median_10m,hr_rest_rate_10m,hr_moderate_rate_10m,hr_very_active_rate_10m,SDNN_5m,pHR2_5m,rMSSD_5m,low_hr_5m,high_hr_5m,l_h_5m,CR_5m,hr_mean_5m,hr_var_5m,hr_std_5m,hr_median_5m,hr_rest_rate_5m,hr_moderate_rate_5m,hr_very_active_rate_5m
0,110.0,0.0,0.0,110.0,0.0,4.311111,209.813284,0.0,0.155556,0.100000,0.066667,0.044444,76.0,0.0,2.216667,133.325141,0.0,0.050000,0.033333,0.033333,0.033333,76.0,0.0,4.133333,260.947126,0.0,0.066667,0.066667,0.066667,0.066667,76.0,0.0,12.4,726.933333,0.0,0.2,0.2,0.2,0.2,76.0,...,1.844570,69.0,99.0,0.696970,0.779528,78.654545,37.800665,6.148225,77.0,0.539394,0.121212,0.000000,0.001125,0.186441,1.978700,71.0,99.0,0.717172,0.779528,81.916667,61.806497,7.861711,80.0,0.316667,0.233333,0.000000,0.001094,0.125000,1.968502,76.0,99.0,0.767677,0.779528,85.757576,68.189394,8.257687,82.0,0.181818,0.424242,0.000000
1,111.0,0.0,0.0,70.0,0.0,1.205556,60.074829,0.0,0.044444,0.033333,0.016667,0.011111,21.0,0.0,1.100000,17.922034,0.0,0.066667,0.066667,0.016667,0.000000,21.0,0.0,1.800000,30.648276,0.0,0.100000,0.100000,0.033333,0.000000,21.0,0.0,5.4,77.155556,0.0,0.3,0.3,0.1,0.0,0.0,...,2.251811,50.0,103.0,0.485437,0.811024,60.913514,150.818566,12.280821,57.0,0.675676,0.151351,0.075676,0.003763,0.188406,3.180557,50.0,103.0,0.485437,0.811024,66.628571,325.019462,18.028296,56.0,0.557143,0.342857,0.200000,0.000643,0.035714,1.224745,50.0,58.0,0.862069,0.456693,53.034483,3.463054,1.860928,53.0,1.000000,0.000000,0.000000
2,111.0,0.0,0.0,102.0,0.0,18.950000,417.265642,14.5,0.672222,0.577778,0.377778,0.238889,86.0,0.0,19.566667,434.148023,14.5,0.683333,0.583333,0.400000,0.233333,74.0,0.0,24.466667,484.533333,23.0,0.733333,0.666667,0.533333,0.333333,74.0,0.0,21.8,742.622222,16.0,0.6,0.6,0.4,0.2,74.0,...,2.237032,66.0,117.0,0.564103,0.921260,88.798283,154.722066,12.438732,86.0,0.000000,0.952790,0.446352,0.001028,0.166667,2.455153,66.0,91.0,0.725275,0.716535,77.589041,38.745434,6.224583,77.0,0.000000,0.849315,0.054795,0.001182,0.181818,2.037527,66.0,91.0,0.725275,0.716535,76.852941,51.947415,7.207456,75.0,0.000000,0.764706,0.117647
3,111.0,0.0,0.0,72.0,0.0,3.750000,124.613128,0.0,0.166667,0.105556,0.061111,0.038889,72.0,0.0,3.250000,193.648305,0.0,0.066667,0.050000,0.050000,0.050000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,1.046422,65.0,82.0,0.792683,0.625954,71.228856,10.887363,3.299600,71.0,0.000000,0.885572,0.000000,0.000624,0.027778,1.201850,65.0,82.0,0.792683,0.625954,72.178082,10.870624,3.297063,72.0,0.000000,0.931507,0.000000,0.000522,0.054054,1.414214,69.0,82.0,0.841463,0.625954,74.000000,8.648649,2.940858,73.5,0.000000,1.000000,0.000000
4,111.0,0.0,0.0,87.0,0.0,26.577778,496.803973,22.5,0.766667,0.727778,0.550000,0.383333,87.0,0.0,19.100000,535.345763,14.0,0.550000,0.516667,0.366667,0.266667,31.0,0.0,2.900000,61.610345,0.0,0.133333,0.133333,0.066667,0.033333,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,1.242368,77.0,111.0,0.693694,0.847328,85.583784,77.168214,8.784544,82.0,0.000000,1.000000,0.194595,0.000255,0.000000,0.820413,79.0,85.0,0.929412,0.648855,81.320755,2.914369,1.707152,81.0,0.000000,1.000

#### SVM

In [0]:
### SVM Regression
reg_metrics_SVR, predict_metric_SVR, all_values_SVR = group_shuffle(model_SVR, X_ALL_DATA, Y_TARGET, gss)

ValueError: ignored

In [0]:
reg_metrics_SVR.mean()

MAE     0.088839
MSE     0.023299
RMSE    0.149707
dtype: float64

In [0]:
predict_metric_SVR.mean()

recall       0.896042
precision    0.995346
accuracy     0.985245
f1           0.942938
dtype: float64

#### Descriptive data


In [0]:
X_ALL_DATA.shape

(1873, 180)